In [ ]:
#This tutorial runs pytorch neural network example
import torch

# Use torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=torch.device("cpu"), dtype=torch.float) # vs np.random.randn(N, D_in)
y = torch.randn(N, D_out, device=torch.device("cpu"), dtype=torch.float)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=torch.device("cpu"), dtype=torch.float)
w2 = torch.randn(H, D_out, device=torch.device("cpu"), dtype=torch.float)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1) # vs h = x.dot(w1)
    h_relu = h.clamp(min=0) # vs h_relu = np.maximum(h, 0)
    y_pred = h_relu.mm(w2) # vs  y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item() # vs loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred) # vs grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t()) # vs grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.clone() # vs grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h) # vs grad_w1 = x.T.dot(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 612.0463256835938
199 4.462260723114014
299 0.05373704433441162
399 0.0010010827099904418
499 9.729719022288918e-05


In [ ]:
#This tutorial runs neural network example using numpy and without pytorch
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
      print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0 # for all -ve h set grad_h to zero
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 971.217399355822
199 5.3713751270405234
299 0.0439001185282819
399 0.0004454981077200312
499 5.2531391249588826e-06


In [ ]:
# This tutorial shows how to subclass nn.Module to define a custom CNN class
import torch
import numpy as np
from torch.autograd import Variable
from torch.nn import Linear
import torch.nn as nn
import matplotlib.pyplot as plt
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
# This tutorial how to use torch.autograd and torch.nn to build neural network
import torch
import numpy as np
from torch.autograd import Variable
from torch.nn import Linear
import torch.nn as nn
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.float

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential (
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
#loss fn
loss_fn  = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for i in range(500):
  # do forward prop
  y_pred = model(x)

  # calculate loss 
  loss = loss_fn(y_pred, y)

  # print loss
  if i % 100 == 0:
    print(i, loss.item())

  # zero the grads out
  optimizer.zero_grad()

  # compute grad of loss w.r.t all learnable model params
  loss.backward() 

  # update the learnable params
  optimizer.step()


0 625.8773803710938
100 3.5260252952575684
200 0.06934116780757904
300 0.001994086429476738
400 6.989714893279597e-05


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#uploaded = files.upload()
#df2 = pd.read_csv(io.BytesIO(uploaded['wine.csv']))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
dataset = MyWineDataset('wine.csv')
first_data = dataset[0]
features, labels = first_data
print(features, labels)

#load whole dataset with dataloader
# batch size is 4
# shuffle
# num workers

batch_size = 4
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                         shuffle=True, num_workers=2)

#convert to an iterator and look at one random sample
dataiter = iter(train_loader)
data = dataiter.next()
features, labels = data
print(features, labels)

# dummy training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/batch_size)

for epoch in range(num_epochs):
  for i, (inputs, labels) in enumerate(train_loader):
    if ((i+1)%5 == 0):
      print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')
  


In [ ]:
# load some famous datasets
# e.g. MNIST, Fashion-MNIST, CIFAR10, COCO


famous_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=torchvision.transforms.ToTensor(), download=True)

batch_size = 4
train_loader2 = DataLoader(dataset=famous_dataset, batch_size=batch_size, 
                         shuffle=True, num_workers=2)



In [ ]:
# load a random sample
dataiter2 = iter(train_loader2)
data2 = dataiter2.next()
inputs2, outputs2 = data2
print(inputs2, outputs2)
print(inputs2.shape, outputs2.shape)

In [ ]:
# this tutorial demoes a simple CNN
# https://algorithmia.com/blog/convolutional-neural-nets-in-pytorch
import torch
from torch.autograd import Variable
import torch.nn.functional as F

class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 10)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 16 *16)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

def outputSize(in_size, kernel_size, stride, padding):
  output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
  return(output)

In [ ]:
import torch

In [ ]:
t = torch.tensor([[1,2], [3, 4]])
t
print(t.reshape(-1))

tensor([1, 2, 3, 4])


In [ ]:
t.unsqueeze(dim=0)
t

tensor([[1, 2],
        [3, 4]])

In [ ]:
t = torch.randn(2, 4, 3, dtype=torch.float64) 
print(t)
print(t.size())

tensor([[[ 2.0809,  1.6009,  1.0557],
         [ 0.2655, -0.3276,  0.3141],
         [ 1.8768,  0.9406,  0.0193],
         [-1.1189, -1.9978,  0.9711]],

        [[ 1.2631,  1.5536, -0.7883],
         [ 1.0088, -0.5322, -2.2212],
         [ 0.3917,  0.4168,  0.3823],
         [-0.5199, -0.3925, -0.6523]]], dtype=torch.float64)
torch.Size([2, 4, 3])


In [ ]:
x  = t.unsqueeze(2)
print(x.size())
print(x)
y = x.squeeze()
print(y.size())

torch.Size([2, 4, 1, 3])
tensor([[[[ 2.0809,  1.6009,  1.0557]],

         [[ 0.2655, -0.3276,  0.3141]],

         [[ 1.8768,  0.9406,  0.0193]],

         [[-1.1189, -1.9978,  0.9711]]],


        [[[ 1.2631,  1.5536, -0.7883]],

         [[ 1.0088, -0.5322, -2.2212]],

         [[ 0.3917,  0.4168,  0.3823]],

         [[-0.5199, -0.3925, -0.6523]]]], dtype=torch.float64)
torch.Size([2, 4, 3])


In [ ]:
t1 = torch.tensor([1, 1, 1])
t2 = torch.tensor([2, 2, 2])
t3 = torch.tensor([3, 3,3 ])
t = torch.cat((t1, t2, t3))
print(t)
t = torch.stack((t1, t2, t3))
print(t)
print((t1.unsqueeze(0), t2.unsqueeze(0), t3.unsqueeze(0)))
t = torch.cat((t1.unsqueeze(0), t2.unsqueeze(0), t3.unsqueeze(0)), dim=1)
print(t)
t = torch.stack((t1.unsqueeze(0), t2.unsqueeze(0), t3.unsqueeze(0)))
print(t)

tensor([1, 1, 1, 2, 2, 2, 3, 3, 3])
tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])
(tensor([[1, 1, 1]]), tensor([[2, 2, 2]]), tensor([[3, 3, 3]]))
tensor([[1, 1, 1, 2, 2, 2, 3, 3, 3]])
tensor([[[1, 1, 1]],

        [[2, 2, 2]],

        [[3, 3, 3]]])
